# Hellas Fraud Game ABM - Experimental Analysis

This notebook demonstrates the ABM simulator for the Hellas fraud game protocol.

**Reference:** *"Fraud Game Analysis for Hellas Protocol"* (CryptoEconLab, 2026)

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from config import SimulationConfig, ProtocolParameters
from core.engine import SimulationEngine
from analysis.metrics import compute_theoretical_equilibrium, parameter_sensitivity_analysis
from analysis.visualization import (
    plot_simulation_results, 
    plot_equilibrium_analysis,
    plot_parameter_sensitivity,
    plot_attack_comparison
)
from attacks.scenarios import (
    ReputationFarmingAttack,
    NoStakeFloorAttack,
    CensorshipAttack,
    run_all_attacks
)

## 1. Theoretical Equilibrium Analysis

First, let's compute the theoretical equilibrium values from the paper.

In [ ]:
# Default protocol parameters
protocol = ProtocolParameters(
    S_P_min=100.0,     # Minimum provider stake
    c_H=5.0,           # Honest execution cost
    c_F=0.5,           # Cheating cost
    C_safe=8.0,        # Safe fallback cost
    c_proof=2.0,       # Proof generation cost
    c_tx=1.0,          # Transaction cost
    beta=0.5,          # Slash reward fraction
    lambda_=1.0,       # Payment routing fraction
    p_w=0.95,          # Enforcement probability
    B_C=5.0,           # Challenge bond
    L_base=50.0,       # Client loss from fraud
)

# Compute equilibrium
P_set = 50.0
S_P = 100.0
L = 50.0

eq = compute_theoretical_equilibrium(protocol, P_set, S_P, L)

print("=" * 50)
print("THEORETICAL EQUILIBRIUM VALUES")
print("=" * 50)
print(f"""\nMixed Strategy Equilibrium (Theorem 2):
  v* = (c_H - c_F) / (P_set + S_P) = {eq.v_star:.6f}
  q* = C_safe / (L + Δ)           = {eq.q_star:.6f}

Key Parameters:
  θ  (IC threshold)               = {eq.theta:.6f}
  Δ  (dispute surplus)            = {eq.Delta:.2f}
  S_P^min (min viable stake)      = {eq.S_P_min:.2f}
  μ* (belief threshold)           = {eq.mu_star:.6f}

Viability:
  Enforcement viable (S_P >= S_P^min): {eq.is_enforcement_viable}
  IC satisfied (v* >= θ):              {eq.is_ic_satisfied}

Expected Utilities:
  E[U_P] = {eq.expected_provider_utility:.2f}
  E[U_C] = {eq.expected_client_utility:.2f}
""")

In [ ]:
# Visualize IC region
fig = plot_equilibrium_analysis(protocol, L=L)
fig.show()

## 2. Baseline Simulation

Run a baseline simulation and compare to theoretical predictions.

In [ ]:
# Configure simulation
config = SimulationConfig(
    n_providers=50,
    n_clients=100,
    n_challengers=10,
    n_periods=500,
    seed=42,
    jobs_per_period=20,
    job_value_mean=50.0,
    provider_honest_frac=0.6,
    provider_rational_frac=0.3,
    provider_adversarial_frac=0.1,
    protocol=protocol,
)

# Run simulation
engine = SimulationEngine(config)
result = engine.run(show_progress=True)

print("\n" + "=" * 50)
print("SIMULATION RESULTS")
print("=" * 50)
print(f"""\nAggregate Metrics:
  Total Jobs:        {result.total_jobs}
  Total Frauds:      {result.total_frauds}
  Fraud Rate:        {result.fraud_rate:.4f} (theoretical q* = {eq.q_star:.4f})
  Detection Rate:    {result.detection_rate:.4f}
  Stake Slashed:     ${result.total_stake_slashed:.2f}
  Client Losses:     ${result.total_client_losses:.2f}
  Social Welfare:    ${result.social_welfare:.2f}

Attack Metrics:
  Adversarial Profit: ${result.attack_metrics.get('adversarial_total_profit', 0):.2f}
  Honest Mean Profit: ${result.attack_metrics.get('honest_mean_profit', 0):.2f}
  Attack Success Ratio: {result.attack_metrics.get('attack_success_ratio', 0):.2f}x
""")

In [ ]:
# Visualize results
fig = plot_simulation_results(result, title="Baseline Simulation Results")
fig.show()

## 3. Reputation Farming Attack

Analyze the reputation farming vulnerability: providers build reputation with low-stake honest jobs, then exploit reduced auditing.

In [ ]:
print("Running Reputation Farming Attack Analysis...")
print("-" * 50)

# Scenario 1: With minimum stake enforcement (secure)
attack_secure = ReputationFarmingAttack(
    config,
    min_stake_enforced=True,
    farming_periods=50,
)
result_secure = attack_secure.run(n_periods=300, show_progress=False)

# Scenario 2: Without minimum stake enforcement (vulnerable)
attack_vulnerable = ReputationFarmingAttack(
    config,
    min_stake_enforced=False,
    farming_periods=50,
)
result_vulnerable = attack_vulnerable.run(n_periods=300, show_progress=False)

print(f"""\nScenario 1: Min Stake ENFORCED
  Baseline Fraud Rate: {result_secure.baseline_result.fraud_rate:.4f}
  Attack Fraud Rate:   {result_secure.attack_result.fraud_rate:.4f}
  Fraud Rate Increase: {result_secure.fraud_rate_increase:+.4f}
  Welfare Loss:        ${result_secure.social_welfare_loss:.2f}

Scenario 2: Min Stake NOT ENFORCED (Vulnerable)
  Baseline Fraud Rate: {result_vulnerable.baseline_result.fraud_rate:.4f}
  Attack Fraud Rate:   {result_vulnerable.attack_result.fraud_rate:.4f}
  Fraud Rate Increase: {result_vulnerable.fraud_rate_increase:+.4f}
  Welfare Loss:        ${result_vulnerable.social_welfare_loss:.2f}

Key Insight:
  Minimum stake enforcement reduces attack effectiveness by:
  {abs(result_vulnerable.fraud_rate_increase - result_secure.fraud_rate_increase):.2%} fraud rate reduction
""")

## 4. Parameter Sensitivity Analysis

How do equilibrium outcomes change with different protocol parameters?

In [ ]:
# Sensitivity to minimum stake
fig1 = plot_parameter_sensitivity(
    protocol, 
    "S_P_min",
    np.linspace(10, 300, 30).tolist(),
    P_set=50, S_P=100, L=50,
    title="Sensitivity to Minimum Stake (S_P^min)"
)
fig1.show()

In [ ]:
# Sensitivity to enforcement probability
fig2 = plot_parameter_sensitivity(
    protocol,
    "p_w",
    np.linspace(0.5, 1.0, 30).tolist(),
    P_set=50, S_P=100, L=50,
    title="Sensitivity to Enforcement Probability (p_w)"
)
fig2.show()

In [ ]:
# Sensitivity to verification cost
fig3 = plot_parameter_sensitivity(
    protocol,
    "C_safe",
    np.linspace(1, 30, 30).tolist(),
    P_set=50, S_P=100, L=50,
    title="Sensitivity to Verification Cost (C_safe)"
)
fig3.show()

## 5. All Attack Scenarios Comparison

Compare all attack vectors to understand which poses the greatest threat.

In [ ]:
print("Running all attack scenarios (this may take a few minutes)...")
all_attack_results = run_all_attacks(config, n_periods=200, show_progress=True)

In [ ]:
# Summary table
attack_summary = []
for name, result in all_attack_results.items():
    attack_summary.append({
        "Attack": name,
        "Fraud Rate Δ": f"{result.fraud_rate_increase:+.2%}",
        "Detection Δ": f"{result.detection_rate_change:+.2%}",
        "Attacker Profit": f"${result.attacker_profit:.0f}",
        "Victim Losses": f"${result.victim_losses:.0f}",
        "Welfare Loss": f"${result.social_welfare_loss:.0f}",
    })

df = pd.DataFrame(attack_summary)
print("\nAttack Scenario Comparison:")
print("=" * 80)
print(df.to_string(index=False))

In [ ]:
# Visualize attack comparison
fig = plot_attack_comparison(all_attack_results, title="Attack Scenario Comparison")
fig.show()

## 6. Key Economic Insights

### Proposition 1: Provider Incentive Compatibility

A provider prefers honest execution over cheating if and only if:

$$p_d \geq \theta := \frac{c_H - c_F}{P_{set} + S_P}$$

### Proposition 2: Minimum Viable Stake

Disputing is strictly optimal for the challenger if and only if:

$$S_P \geq S_P^{min} := \max\left\{0, \frac{C_{disp} + (1-p_w)B_C - p_w\lambda P_{set}}{p_w\beta}\right\}$$

### Theorem 2: Mixed Equilibrium

The unique mixed-strategy Nash equilibrium has:

$$v^* = \frac{c_H - c_F}{P_{set} + S_P}, \quad q^* = \frac{C_{safe}}{L + \Delta}$$

In [ ]:
# Demonstrate minimum viable stake computation
print("Minimum Viable Stake Analysis")
print("=" * 50)

P_set_values = [20, 50, 100, 200]
for P_set in P_set_values:
    S_P_min = protocol.compute_S_P_min_viable(P_set)
    eq_local = compute_theoretical_equilibrium(protocol, P_set, max(S_P_min, 1), L)
    
    print(f"\nP_set = ${P_set}:")
    print(f"  S_P^min = ${S_P_min:.2f}")
    print(f"  q* at S_P^min = {eq_local.q_star:.4f}")
    print(f"  Stake/Payment ratio = {S_P_min/P_set:.2f}x")

## 7. Protocol Design Recommendations

Based on the analysis, here are key recommendations:

1. **Enforce stake floors indexed by job class** - Prevents the "no stake floor" attack

2. **Stake-weight reputation gains** - Mitigates reputation farming by making it expensive

3. **Enable permissionless challenging** - Creates watcher markets for external auditing

4. **Implement randomized protocol audits** - Provides baseline detection without client burden

5. **Minimize verification cost C_safe** - This is the most impactful lever for reducing q*

6. **Maintain high enforcement reliability p_w** - Critical for deterrence

In [ ]:
# Quantify the impact of reducing C_safe
print("Impact of Reducing Verification Cost (C_safe)")
print("=" * 50)

C_safe_values = [2, 5, 8, 15, 25]
for C_safe in C_safe_values:
    protocol_mod = ProtocolParameters(**{**protocol.__dict__, 'C_safe': C_safe})
    eq_local = compute_theoretical_equilibrium(protocol_mod, P_set=50, S_P=100, L=50)
    
    print(f"\nC_safe = ${C_safe}:")
    print(f"  q* = {eq_local.q_star:.4f}")
    print(f"  Expected client loss = ${eq_local.q_star * 50 * (1 - eq_local.v_star):.2f}")

## Conclusion

This ABM simulator validates the theoretical equilibrium predictions from the Hellas fraud game analysis and provides quantitative insights into attack vulnerabilities.

**Key Findings:**
1. Simulated fraud rates converge to theoretical q* predictions
2. Reputation farming is most effective without stake floors
3. Reducing C_safe is the most impactful lever for improving outcomes
4. Protocol design should prioritize stake floors and enforcement reliability